# RAG-Pipeline (Retrieval-Augmented Generation) mit ChromaDB und FLAN-T5

Large Language Models (LLMs) haben oft das Problem, dass ihr Wissen auf den Trainingszeitpunkt beschränkt ist und sie keine aktuellen oder domänenspezifischen Informationen abrufen können. Eine Retrieval-Augmented Generation (RAG)-Pipeline löst dieses Problem, indem sie externe Wissensquellen nutzt: Statt sich ausschließlich auf das Modellgedächtnis zu verlassen, durchsucht RAG eine Datenbank nach relevanten Informationen und kombiniert diese mit der generierten Antwort.

In dieser Übung setzen wir eine RAG-Pipeline mit ChromaDB und FLAN-T5 um. ChromaDB dient als Vektordatenbank, in der wir Dokumente ablegen und über Embeddings nach relevanten Inhalten suchen können. FLAN-T5 wird als Sprachmodell genutzt, um eine Antwort auf Basis des abgerufenen Kontexts zu generieren. Diese Kombination ermöglicht es, präzisere, kontextbezogene Antworten zu erhalten – ein Ansatz, der insbesondere für Unternehmenswissen, technische Dokumentation oder domänenspezifische Fragen wertvoll ist.

Quelle Abbildung: https://medium.com/@bijit211987/advanced-rag-for-llms-slms-5bcc6fbba411

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*q1CkGPwS7g4-f9rNbPrkig.png">

In [17]:
#!pip install torch transformers langchain langchain-community chromadb sentence-transformers

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from huggingface_hub import notebook_login
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

Die Funktion `notebook_login()` wird verwendet, um sich in einem Hugging Face-Konto (https://huggingface.co) innerhalb eines Jupyter Notebook anzumelden. Dies ist erforderlich, wenn man auf private oder *gated* Modelle, Datasets oder Tokenizer von Hugging Face zugreifen möchte.

>**Wichtig**: Dafür wird ein Hugging Face-Account benötigt. Zugangstokens können im Profil unter 'Access Tokens' erstellt werden.

In [14]:
notebook_login()

Dieser Code erstellt eine Vektordatenbank mit ChromaDB, die semantische Suchen ermöglicht. Dadurch können später inhaltlich ähnliche Texte effizient gefunden werden.

`HuggingFaceEmbeddings` wandelt Text in Vektoren um, um diese in der Vektor-DB speichern zu können. Die DB ist im Verzeichnis `./chroma_db` zu finden.

In [18]:
#huggingface_token = "xxx"

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

docs = [
    "Apple hat 2024 einen Umsatz von 400 Milliarden USD gemacht.",
    "Microsoft investiert massiv in Künstliche Intelligenz.",
    "Tesla entwickelt neue Batterietechnologien für Elektroautos."
]
db.add_texts(docs)

['cfd6c9d6-ea73-48c4-b851-803c63d67c4d',
 'c2712105-27e5-4492-8fd2-1d676994e744',
 '6f091b86-44eb-4cf8-b4f2-8c168512aabc']

`db.similarity_search(question, k=1)` sucht in der Vektordatenbank nach dem am besten passenden (semantisch ähnlichsten) Dokument. `k=1` bedeutet, dass nur das relevanteste Dokument zurückgegeben wird.

In [19]:
question = "Wie viel Umsatz hat Apple 2024 gemacht?"
#question = "Investiert Microsoft?"

retrieved_docs = db.similarity_search(question, k=1)

context = retrieved_docs[0].page_content
print("Gefundener Kontext:", context)


Gefundener Kontext: Apple hat 2024 einen Umsatz von 400 Milliarden USD gemacht.


Hier wird das *FLAN-T5-Base-Modell* (Modell-ID: `google/flan-t5-base`) geladen. Muss nur 1x ausgeführt werden.

In [20]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Abschließend muss der Eingabe-Prompt (inklusive Kontext) in Tokens umgewandelt werden, damit der Text von einem LLM verarbeitet werden kann.

In [24]:
#prompt = f"\n\nQuestion: {question}\nAnswer:"
prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"

# Tokenisierung
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print("Antwort:", response)

Antwort: 400 billion USD
